In [1]:
#!pip uninstall libximc -y
#!pip install libximc --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 19.0/19.0 MB 13.1 MB/s eta 0:00:00


In [4]:
import libximc
print("Connecting the libximc library OK")
libximc.lib

Connecting the libximc library OK


<WinDLL 'C:\Users\e0974196.NUSSTU\AppData\Roaming\Python\Python39\site-packages\libximc\win64\libximc.dll', handle 180000000 at 0x272b6c91f40>

In [5]:
# Connecting additional dependencies
import tempfile
import urllib
import re
import time
import matplotlib.pyplot as plt
%matplotlib inline
from libximc import *
from ctypes import *
print("Connecting additional dependencies")

Connecting additional dependencies


In [6]:
# variable 'lib' points to a loaded library
# note that ximc uses stdcall on win
print("Library loaded")
sbuf = create_string_buffer(64)
lib.ximc_version(sbuf)
print("Library version: " + sbuf.raw.decode().rstrip("\0"))

Library loaded
Library version: 2.13.6


In [7]:
# Set name a virtual controller
tempdir = tempfile.gettempdir() + "/testdevice.bin"
uri = urllib.parse.urlunparse(urllib.parse.ParseResult(scheme="file", \
            netloc=None, path=tempdir, params=None, query=None, fragment=None))
open_name = re.sub(r'^file', 'xi-emu', uri).encode()

In [8]:
# Open controller
tupevirt = 1;
device_id = lib.open_device(open_name)
if device_id > 0:
    print(b"The controller is open/" + open_name)
    print("Device id: " + repr(device_id))
    if 'xi-emu' in str(open_name):
        tupevirt = 0;
else:
    print(b"It was not possible to open the " + open_name + b" libximc controller")

b'The controller is open/xi-emu:///C:\\Users\\E09741~1.NUS\\AppData\\Local\\Temp/testdevice.bin'
Device id: 1


In [9]:
# Output of information about the device
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:    
    x_device_information = device_information_t()
    result = lib.get_device_information(device_id, byref(x_device_information))
    if result == Result.Ok:
            print("Device information:")
            print(" Manufacturer: " +
                    repr(string_at(x_device_information.Manufacturer).decode()))
            print(" ManufacturerId: " +
                    repr(string_at(x_device_information.ManufacturerId).decode()))
            print(" ProductDescription: " +
                    repr(string_at(x_device_information.ProductDescription).decode()))
            print(" Major: " + repr(x_device_information.Major))
            print(" Minor: " + repr(x_device_information.Minor))
            print(" Release: " + repr(x_device_information.Release))

Device information:
 Manufacturer: 'XIMC'
 ManufacturerId: 'SM'
 ProductDescription: 'XISM-USB'
 Major: 0
 Minor: 0
 Release: 0


In [10]:
# Output serial number
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else: 
    print("\nReading serial")
    x_serial = c_uint()
    result = lib.get_serial_number(device_id, byref(x_serial))
    if result == Result.Ok:
        print("Serial: " + repr(x_serial.value))


Reading serial
Serial: 0


In [11]:
# Output firmware version
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else: 
    print("\nReading firmware version")
    x_Major = c_uint()
    x_Minor = c_uint()
    x_Release = c_uint()
    result = lib.get_firmware_version (device_id, byref(x_Major), byref(x_Minor), byref(x_Release));
    if result == Result.Ok:
        print("Major: " + repr(x_Major.value))
        print("Minor: " + repr(x_Minor.value))
        print("Release: " + repr(x_Release.value))


Reading firmware version
Major: 0
Minor: 0
Release: 0


In [12]:
# Basic configuration of user units
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:
    print("Setting user units by default")
    
# Create engine settings structure
eng = engine_settings_t()
result = lib.get_engine_settings(device_id, byref(eng))

# Create user unit settings structure
user_unit = calibration_t()
user_unit.MicrostepMode = eng.MicrostepMode
user_unit.A = 1;

Setting user units by default


In [13]:
# A set of basic functions
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:
    print("Loading a set of basic functions")


def input_new(s=""):
    """
    Wrapper for the data entry function.
    
    :param s:  Description of the input value(Default value = "")
    """
    
    return input(s)


def test_user_unit_mode(lib, device_id):
    """
    User unit mode settings
    
    After setting this multiplier, you can use special commands with the suffix _calb to set the movement in mm or degrees.
    Follow the on-screen instructions to change the settings.
    
    :param lib: structure for accessing the functionality of the libximc library.
    :param device_id: device id.
    """

    print("\nUser unit mode settings.")
    print("User unit coordinate multiplier = {0} \n".format(user_unit.A) )
    try:
        fl_val = float(input_new("Set new coordinate multiplier = "))
        if fl_val>0:
            user_unit.A = fl_val
        else:
            print("The multiplier cannot be less than or equal to 0.")
        # user_unit.MicrostepMode the value is set together with eng.MicrostepMode

    except:
        print("User unit coordinate multiplier = ", user_unit.A )

        
def test_get_position(lib, device_id, mode=1):
    """
    Obtaining information about the position of the positioner.
    
    This function allows you to get information about the current positioner coordinates,
    both in steps and in encoder counts, if it is set.
    Also, depending on the state of the mode parameter, information can be obtained in user units.
    
    :param lib: structure for accessing the functionality of the libximc library.
    :param device_id: device id.
    :param mode: mode in feedback counts or in user units. (Default value = 1)
    """
    moveStatus = status_MvCmdSts(lib, device_id)
    # print("\nRead position")
    if mode:
        x_pos = get_position_t()
        result = lib.get_position(device_id, byref(x_pos))
        if result == Result.Ok:
            print("Position: {0} steps, {1} microsteps // Move status {1}, Movr status {2}                 ".format(x_pos.Position, x_pos.uPosition, moveStatus[0], moveStatus[1]),  end="\r")
        return x_pos.Position, x_pos.uPosition
    else:
        x_pos = get_position_calb_t()
        result = lib.get_position_calb(device_id, byref(x_pos), byref(user_unit))
        if result == Result.Ok:
            print("Position: {0} user unit // Move status {1}, Movr status {2}                 ".format(x_pos.Position, moveStatus[0], moveStatus[1]),  end="\r")
        return x_pos.Position, 0

def test_move(lib, device_id, distance, udistance, mode=1):
    """
    Move to the specified coordinate.

    Depending on the mode parameter, you can set coordinates in steps or feedback counts, or in custom units.
    
    :param lib: structure for accessing the functionality of the libximc library.
    :param device_id: device id.
    :param distance: the position of the destination.
    :param udistance: destination position in micro steps if this mode is used.
    :param mode:  mode in feedback counts or in user units. (Default value = 1)
    """
    
    if mode:
        print("\nGoing to {0} steps, {1} microsteps".format(distance, udistance))
        result = lib.command_move(device_id, distance, udistance)
    else:
        # udistance is not used for setting movement in custom units.
        print("\nMove to the position {0} specified in user units.".format(distance))
        result = lib.command_move_calb(device_id, c_float(distance), byref(user_unit))
        
def test_movr(lib, device_id, distance, udistance, mode=1):
    """
    The shift by the specified offset coordinates.
    
    Depending on the mode parameter, you can set coordinates in steps or feedback counts, or in custom units.
    
    :param lib: structure for accessing the functionality of the libximc library.
    :param device_id: device id.
    :param distance: size of the offset in steps.
    :param udistance: Size of the offset in micro steps.
    :param mode:  (Default value = 1)
    """
    
    if mode:
        print("\nShift to {0} steps, {1} microsteps".format(distance, udistance))
        result = lib.command_movr(device_id, distance, udistance)
    else:
        # udistance is not used for setting movement in custom units.
        print("\nShift to the position {0} specified in user units.".format(distance))
        result = lib.command_movr_calb(device_id, c_float(distance), byref(user_unit))

def get_status(lib, device_id):
    """
    A function of reading status information from the device

    You can use this function to get basic information about the device status.
    
    :param lib: structure for accessing the functionality of the libximc library.
    :param device_id:  device id.
    """
    
    x_status = status_t()
    result = lib.get_status(device_id, byref(x_status))
    if result == Result.Ok:
        return x_status
    else:
        return None
    
def status_MvCmdSts_MVCMD_RUNNING(lib, device_id):
    currStatus = get_status(lib, device_id)
    return (currStatus.MvCmdSts & MvcmdStatus.MVCMD_RUNNING) # 0x80) # )

def status_MvCmdSts(lib, device_id):
    currStatus = get_status(lib, device_id)
    """
    MVCMD_MOVE         = 0x01
    MVCMD_MOVR         = 0x02
    MVCMD_LEFT         = 0x03
    MVCMD_RIGHT        = 0x04
    MVCMD_STOP         = 0x05
    MVCMD_HOME         = 0x06
    MVCMD_LOFT         = 0x07
    MVCMD_SSTP         = 0x08
    MVCMD_ERROR        = 0x40
    MVCMD_RUNNING      = 0x80
    """
    return (currStatus.MvCmdSts & MvcmdStatus.MVCMD_MOVE), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_MOVR), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_LEFT),
    (currStatus.MvCmdSts & MvcmdStatus.MVCMD_RIGHT), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_STOP), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_HOME),
    (currStatus.MvCmdSts & MvcmdStatus.MVCMD_LOFT), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_SSTP), (currStatus.MvCmdSts & MvcmdStatus.MVCMD_ERROR),
    (currStatus.MvCmdSts & MvcmdStatus.MVCMD_RUNNING)
    
def status_StateFlags(lib, device_id):
    currStatus = get_status(lib, device_id)
    """
    STATE_IS_HOMED                  = 0x0000020
    STATE_ALARM                     = 0x0000040
    STATE_CTP_ERROR                 = 0x0000080
    STATE_POWER_OVERHEAT            = 0x0000100
    STATE_CONTROLLER_OVERHEAT       = 0x0000200
    STATE_OVERLOAD_POWER_VOLTAGE    = 0x0000400
    STATE_OVERLOAD_POWER_CURRENT    = 0x0000800
    STATE_OVERLOAD_USB_VOLTAGE      = 0x0001000
    STATE_LOW_USB_VOLTAGE           = 0x0002000
    STATE_OVERLOAD_USB_CURRENT      = 0x0004000
    STATE_BORDERS_SWAP_MISSET       = 0x0008000
    STATE_LOW_POWER_VOLTAGE         = 0x0010000
    STATE_H_BRIDGE_FAULT            = 0x0020000
    STATE_WINDING_RES_MISMATCH      = 0x0100000
    STATE_ENCODER_FAULT             = 0x0200000
    STATE_ENGINE_RESPONSE_ERROR     = 0x0800000
    STATE_EXTIO_ALARM               = 0x1000000
    """
    
    print("")
    if (currStatus.Flags & StateFlags.STATE_IS_HOMED):
        print(" True:STATE_IS_HOMED ")
    else:
        print("False:STATE_IS_HOMED ")

    if (currStatus.Flags & StateFlags.STATE_ALARM):
        print(" True:STATE_ALARM")
    else:
        print("False:STATE_ALARM")

    if (currStatus.Flags & StateFlags.STATE_CTP_ERROR):
        print(" True:STATE_CTP_ERROR")
    else:
        print("False:STATE_CTP_ERROR")

    if (currStatus.Flags & StateFlags.STATE_POWER_OVERHEAT):
        print(" True:STATE_POWER_OVERHEAT")
    else:
        print("False:STATE_POWER_OVERHEAT")

    if (currStatus.Flags & StateFlags.STATE_OVERLOAD_POWER_VOLTAGE):
        print(" True:STATE_OVERLOAD_POWER_VOLTAGE")
    else:
        print("False:STATE_OVERLOAD_POWER_VOLTAGE")

    if (currStatus.Flags & StateFlags.STATE_OVERLOAD_POWER_CURRENT):
        print(" True:STATE_OVERLOAD_POWER_CURRENT")
    else:
        print("False:STATE_OVERLOAD_POWER_CURRENT")

    if (currStatus.Flags & StateFlags.STATE_OVERLOAD_USB_VOLTAGE):
        print(" True:STATE_OVERLOAD_USB_VOLTAGE")
    else:
        print("False:STATE_OVERLOAD_USB_VOLTAGE")

    if (currStatus.Flags & StateFlags.STATE_LOW_USB_VOLTAGE):
        print(" True:STATE_LOW_USB_VOLTAGE")
    else:
        print("False:STATE_LOW_USB_VOLTAGE")

    if (currStatus.Flags & StateFlags.STATE_OVERLOAD_USB_CURRENT):
        print(" True:STATE_OVERLOAD_USB_CURRENT")
    else:
        print("False:STATE_OVERLOAD_USB_CURRENT")
    if (currStatus.Flags & StateFlags.STATE_BORDERS_SWAP_MISSET):
        print(" True:STATE_BORDERS_SWAP_MISSET")
    else:
        print("False:STATE_BORDERS_SWAP_MISSET")

    if (currStatus.Flags & StateFlags.STATE_LOW_POWER_VOLTAGE):
        print(" True:STATE_LOW_POWER_VOLTAGE")
    else:
        print("False:STATE_LOW_POWER_VOLTAGE")

    if (currStatus.Flags & StateFlags.STATE_H_BRIDGE_FAULT):
        print(" True:STATE_H_BRIDGE_FAULT")
    else:
        print("False:STATE_H_BRIDGE_FAULT")
        
    if (currStatus.Flags & StateFlags.STATE_WINDING_RES_MISMATCH):
        print(" True:STATE_WINDING_RES_MISMATCH")
    else:
        print("False:STATE_WINDING_RES_MISMATCH")
        
    if (currStatus.Flags & StateFlags.STATE_ENCODER_FAULT):
        print(" True:STATE_ENCODER_FAULT")
    else:
        print("False:STATE_ENCODER_FAULT")
        
    if (currStatus.Flags & StateFlags.STATE_ENGINE_RESPONSE_ERROR):
        print(" True:STATE_ENGINE_RESPONSE_ERROR")
    else:
        print("False:STATE_ENGINE_RESPONSE_ERROR")
        
    if (currStatus.Flags & StateFlags.STATE_EXTIO_ALARM):
        print(" True:STATE_EXTIO_ALARM")
    else:
        print("False:STATE_EXTIO_ALARM")
        
def plot_move(deltatime):
    x = []
    y = []
    time1 = 0;
    while status_MvCmdSts_MVCMD_RUNNING(lib, device_id):
        x.append(time1)
        y.append(test_get_position(lib, device_id, userMode)[0])
        # print(y)
        time1 += deltatime 
        time.sleep(deltatime)
        if tupevirt:
            plt.plot(x, y)
            


Loading a set of basic functions


In [14]:
# Customizing User Units
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:
    userMode = int(input_new("Set user unit mode 1-steps, 0-user units = "))
    if userMode == 0:
        test_user_unit_mode(lib, device_id)

Set user unit mode 1-steps, 0-user units = 1


In [15]:
# Move to a given position
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:
    if userMode:
        position = int(input_new("Set the 'Move to' position in step mode = "))
    else:
        position = int(input_new("Set the 'Move to' position in user units mode = "))

    test_move(lib, device_id, position, 0, userMode)

    plot_move(0.125)

    status_StateFlags(lib, device_id)

Set the 'Move to' position in step mode = 1

Going to 1 steps, 0 microsteps
Position: 1 steps, 0 microsteps // Move status 0, Movr status 1                 
False:STATE_IS_HOMED 
False:STATE_ALARM
False:STATE_CTP_ERROR
False:STATE_POWER_OVERHEAT
False:STATE_OVERLOAD_POWER_VOLTAGE
False:STATE_OVERLOAD_POWER_CURRENT
False:STATE_OVERLOAD_USB_VOLTAGE
False:STATE_LOW_USB_VOLTAGE
False:STATE_OVERLOAD_USB_CURRENT
False:STATE_BORDERS_SWAP_MISSET
False:STATE_LOW_POWER_VOLTAGE
False:STATE_H_BRIDGE_FAULT
False:STATE_WINDING_RES_MISMATCH
False:STATE_ENCODER_FAULT
False:STATE_ENGINE_RESPONSE_ERROR
False:STATE_EXTIO_ALARM


In [16]:
# Shift on by a given amount
if device_id<=0:
    print("Before performing actions with the controller, it must be selected from one of the blocks 2-4 and opened in block 5.")
else:
    if userMode:
        offset = int(input_new("Set the 'Move to' position in step mode = "))
    else:
        offset = int(input_new("Set the 'Shift on' position in user units mode = "))

    test_movr(lib, device_id, offset,  0, userMode)
    plot_move(0.125)

    status_StateFlags(lib, device_id)
    

Set the 'Move to' position in step mode = 1

Shift to 1 steps, 0 microsteps
Position: 2 steps, 0 microsteps // Move status 0, Movr status 0                 
False:STATE_IS_HOMED 
False:STATE_ALARM
False:STATE_CTP_ERROR
False:STATE_POWER_OVERHEAT
False:STATE_OVERLOAD_POWER_VOLTAGE
False:STATE_OVERLOAD_POWER_CURRENT
False:STATE_OVERLOAD_USB_VOLTAGE
False:STATE_LOW_USB_VOLTAGE
False:STATE_OVERLOAD_USB_CURRENT
False:STATE_BORDERS_SWAP_MISSET
False:STATE_LOW_POWER_VOLTAGE
False:STATE_H_BRIDGE_FAULT
False:STATE_WINDING_RES_MISMATCH
False:STATE_ENCODER_FAULT
False:STATE_ENGINE_RESPONSE_ERROR
False:STATE_EXTIO_ALARM


In [17]:
# Emergency stop with alarm reset
lib.command_stop(device_id)
print("Emergency stop with alarm reset")


Emergency stop with alarm reset


In [18]:
print("\nClosing")

# The device_t device parameter in this function is a C pointer, unlike most library functions that use this parameter
lib.close_device(byref(cast(device_id, POINTER(c_int))))
device_id = -1;
print("Done")


Closing
Done
